# Final Retrieval System

This is a (slightly) improved retrieval system based on the BM25 retrieval.
It implements a custom index, query extension and stemming, sequential dependence and a simple pipeline.

### Step 1: Import Libraries

Imports TIRA, PyTerrier and Spacy and ensures PyTerrier is loaded.

In [11]:
from tira.third_party_integrations import ensure_pyterrier_is_loaded, persist_and_normalize_run

# PyTerrier must be imported after the call to ensure_pyterrier_is_loaded in TIRA.
import pyterrier as pt

import spacy

import nltk
nltk.download('wordnet')
from nltk.corpus import wordnet

# Ensure PyTerrier is loaded
ensure_pyterrier_is_loaded()


[nltk_data] Downloading package wordnet to /root/nltk_data...


### Step 2: Setup for Stopwords

Loads English stopwords from Spacy. Saves them in the file "stopwords.txt" and changes the PyTerrier stopwords file to that. 

In [8]:
# Load Spacy and set up stopwords
nlp = spacy.load("en_core_web_sm")
spacy_stopwords = set(nlp.Defaults.stop_words)


file_path = "stopwords.txt"
with open(file_path, 'w+') as file:
    for element in spacy_stopwords:
        file.write(element+ "\n")

pt.set_property('stopwords.filename','./stopwords.txt')

### Step 3: Import Dataset

Imports the ir_dataset from TIRA and prints the first two queries to see if it worked.

In [9]:
data = pt.get_dataset('irds:ir-lab-sose-2024/ir-acl-anthology-20240504-training')
print('See the first two queries:')
topics = data.get_topics('title')
print(topics.head(2))

See the first two queries:
Download from the Incubator: https://files.webis.de/data-in-production/data-research/tira-zenodo-dump-preparation/ir-lab-sose2024/ir-acl-anthology-20240504-truth.zip?download=1
	This is only used for last spot checks before archival to Zenodo.


Download: 100%|██████████| 29.6k/29.6k [00:00<00:00, 1.83MiB/s]

Download finished. Extract...
Extraction finished:  /root/.tira/extracted_datasets/ir-lab-sose-2024/ir-acl-anthology-20240504-training/
  qid                                     query
0   1  retrieval system improving effectiveness
1   2  machine learning language identification


### Step 4: adding synonym recognition

In [12]:
nltk.download('wordnet')

def get_synonyms(word):
    synonyms = set()
    for syn in wordnet.synsets(word):
        for lemma in syn.lemmas():
            synonyms.add(lemma.name())
    return synonyms

def expand_with_synonyms(text):
    words = text.split()
    expanded_text = []
    for word in words:
        expanded_text.append(word)
        expanded_text.extend(get_synonyms(word))
    return " ".join(expanded_text)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Function to lemmatize text

In [ ]:
# Function to lemmatize text
def lemmatize_text(text):
    doc = nlp(text)
    lemmatize_text = " ".join([token.lemma_ for token in doc if token.text.lower() not in spacy_stopwords ])
    return expand_with_synonyms(lemmatized_text)



### Step 5: Build an Index with lemmatization and and synonym recognition

Builds a custom iterable index. 
Stems words and removes stopwords previously defined.
Block indexing is enabled.

In [13]:
class LemmaSynonymIndexer(pt.IterDictIndexer):
    def process(self, doc):
        doc['text'] = lemmatize_and_expand_text(doc['text'])
        return doc

indexer = LemmaSynonymIndexer("/tmp/index", overwrite=True, blocks=True, meta={'docno': 100, 'text': 20480})
index_ref = indexer.index(data.get_corpus_iter())
print('Done. Index is created')

Download from the Incubator: https://files.webis.de/data-in-production/data-research/tira-zenodo-dump-preparation/ir-lab-sose2024/ir-acl-anthology-20240504-inputs.zip?download=1
	This is only used for last spot checks before archival to Zenodo.


Download: 100%|██████████| 39.4M/39.4M [00:00<00:00, 70.0MiB/s]


Download finished. Extract...
Extraction finished:  /root/.tira/extracted_datasets/ir-lab-sose-2024/ir-acl-anthology-20240504-training/


ir-lab-sose-2024/ir-acl-anthology-20240504-training documents:  66%|██████▌   | 83176/126958 [00:24<00:10, 4119.22it/s]

11:57:01.256 [ForkJoinPool-1-worker-3] WARN org.terrier.structures.indexing.Indexer - Adding an empty document to the index (1980.sigirconf_conference-80.24) - further warnings are suppressed


ir-lab-sose-2024/ir-acl-anthology-20240504-training documents: 100%|██████████| 126958/126958 [00:31<00:00, 4056.47it/s] 


11:57:13.199 [ForkJoinPool-1-worker-3] WARN org.terrier.structures.indexing.Indexer - Indexed 4 empty documents


KeyboardInterrupt: 

### Step 6: Setup for Retrieval-Pipeline

First definitions of index and retrieval models used.
"sdm" enables use of Sequential Dependance Model.

In [ ]:
index = pt.IndexFactory.of(index_ref)
bm25 = pt.BatchRetrieve(index, wmodel="BM25", k1=1.5, b=0.75, verbose=True)
pl2 = pt.BatchRetrieve(index, wmodel="PL2", c=7.0, verbose=True)
sdm = pt.rewrite.SequentialDependence()

Secondly, definition of the query expansion and the actual pipeline.

In [ ]:
#Query Expansion
bo1 = pt.rewrite.Bo1QueryExpansion(index) 
rm3 = pt.rewrite.RM3(index)

#Pipeline
pipe = (bm25 % 100) >> rm3 >> sdm >> pl2

### Step 7: Create the Run

Creates the run on the queries from the dataset.

In [ ]:
print('Create run')

run = pipe(topics)

print('Done, run was created')

Create run


BR(BM25):   0%|          | 0/68 [00:00<?, ?q/s]

BR(BM25): 100%|██████████| 68/68 [00:11<00:00,  5.97q/s]


17:50:07.501 [main] WARN org.terrier.querying.RM1 - Did not identify any usable candidate expansion terms from docid 125137 among 6 possibilities
17:50:07.594 [main] WARN org.terrier.querying.RM1 - Did not identify any usable candidate expansion terms from docid 116910 among 5 possibilities


BR(PL2): 100%|██████████| 68/68 [00:13<00:00,  5.00q/s]

Done, run was created


### Step 8: Save the Runfile

In [ ]:
persist_and_normalize_run(run, 'retrieval_system', default_output='../runs')

The run file is normalized outside the TIRA sandbox, I will store it at "../runs".
Done. run file is stored under "../runs/run.txt".
